In [52]:
from dataclasses import dataclass
import csv
import os
from gensim.test.utils import common_texts
from gensim.models import Word2Vec


"""
2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. 
Dictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}

Once we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)
"""

'\n2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. \nDictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}\n\nOnce we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)\n'

In [53]:
@dataclass
class Influencer:
    name: str
    affiliation: str
    no_followers: int = 0
    before_corpus: str = ""
    after_corpus: str = ""

data = []
account_list = []
before_corpus = ""
after_corpus = ""

#Loading File paths
supplementary_folder = "Supplementary Materials"
raw_data_folder = os.path.join("Raw Data", "Before")
parsed_folder = os.path.join("Parsed Data", "Before")
parsed_run1_folder = os.path.join("Parsed Data", "After", "Run 1")
parsed_run2_folder = os.path.join("Parsed Data", "After", "Run 2")
parsed_before_folder = os.path.join("Parsed Data", "Before")

influencers_path = os.path.join(supplementary_folder, "Followers List & Categories - Accounts Kept.csv")


#Population the data file with initial data of the available influencers
with open(influencers_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) #skip header
    for line in reader:
        name = line[0]
        account_list.append(name[1:]) #Creating a list of influencers account names

        affiliation = line[1]
        if affiliation == " Libertarian Party":
            affiliation = affiliation[1:]

        followers = line[2]
        if affiliation or followers:
            data.append(Influencer(name[1:], affiliation, followers))


for i in range(6):
    csv_file = os.path.join(parsed_run1_folder, f"{i+1} - Parsed_2024-09-22_2023-10-07.csv")
    with open(csv_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) #skip header
        
        for line in reader:
            if not line:  # Skip empty lines
                continue
            if line[0] == "|RUN STATISTICS|":
                continue
        
            name = line[0].strip() if line else ""
            tweet = line[2].strip()
            date = line[1]
            for i in data:
                if name == i.name:
                    if tweet == i.after_corpus:
                        if not tweet.endswith("."):
                            tweet = tweet.strip() + ". "
                        i.after_corpus = i.after_corpus + tweet
                        after_corpus = after_corpus + tweet
for i in range(2):
    csv_file = os.path.join(parsed_run2_folder, f"{i+7} - Parsed_2024-09-22_2023-10-07.csv")
    with open(csv_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) #skip header
        
        for line in reader:
            if not line:  # Skip empty lines
                continue
            if line[0] == "|RUN STATISTICS|":
                continue
        
            name = line[0].strip() if line else ""
            tweet = line[2]
            for i in data:
                if name == i.name:
                    if not tweet.endswith("."):
                            tweet = tweet.strip() + ". "
                    if tweet not in i.after_corpus:
                        i.after_corpus = i.after_corpus + tweet
                        after_corpus = after_corpus + tweet

for i in range(2):
    csv_file = os.path.join(parsed_before_folder, f"{i+1} - Parsed_2023-10-07_2022-10-07.csv")
    with open(csv_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) #skip header
        
        for line in reader:
            if not line:  # Skip empty lines
                continue
            if line[0] == "|RUN STATISTICS|":
                continue
        
            name = line[0].strip() if line else ""
            tweet = line[2].strip()
            for i in data:
                if name == i.name:
                    if not tweet.endswith("."):
                            tweet = tweet.strip() + ". "
                    if tweet not in i.before_corpus:
                        i.before_corpus = i.before_corpus + tweet
                        before_corpus = before_corpus + tweet


with open("Pre-processing.csv", 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Name', 'Affiliation', 'Number of Followers', "Before Corpus", "After Corpus"])
        for i in data:
            if i.before_corpus and i.after_corpus:
                writer.writerow([i.name, i.affiliation, i.no_followers, i.before_corpus, i.after_corpus])

In [54]:
# @dataclass
# class Post:
#     status: str
#     text: str
#     sentiment: str 
#     toxicity: str 
#     adj: str
#     adj_count: int

# all = []

# for post in before: 
#     all.append(Post('Before', post, '', '', '', 0))
# for post in after: 
#     all.append(Post('After', post, '', '', '', 0))

In [55]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# # Define a function to perform sentiment analysis
# def analyze_sentiment(text):
#     # Tokenize input text
#     inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

#     # Perform inference
#     outputs = model(**inputs)
#     logits = outputs.logits

#     # Apply softmax to get probabilities
#     probs = torch.softmax(logits, dim=1).detach().numpy()[0]

#     # Determine sentiment label
#     label_mapping = {
#         0: "Very negative",
#         1: "Negative",
#         2: "Neutral",
#         3: "Positive",
#         4: "Very positive"
#     }
#     sentiment_label = label_mapping[int(probs.argmax())]

#     return sentiment_label, probs

# negative_sentences = []
# positive_sentences = []

# for post in all:
#     # Perform sentiment analysis
#     text = post.text
#     sentiment, probabilities = analyze_sentiment(text)
#     post.sentiment = sentiment


In [56]:
# from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline

# model_path = "JungleLee/bert-toxic-comment-classification"
# tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# # print(pipeline("Whatever happens I love you all and the sun will come up tomorrow. Now lets see those memes!."))

# for post in all:
#     # Perform sentiment analysis
#     text = post.text
#     result = pipeline(text)
#     for r in result: 
#         label, score = r['label'], r['score']
#         post.toxicity = label

In [57]:
# import spacy 
# from spacy.matcher import Matcher

# en_nlp = spacy.load("en_core_web_sm")

# matcher = Matcher(en_nlp.vocab)

# # patterns = [
# #     [{'POS':'ADJ'}, {'POS':'NOUN'}], [{'POS':'AUX'}, {'POS':'ADJ'}], [{'POS':'ADJ'}]
# #     ]
# patterns = [
#     [{'POS':'ADJ'}]
#     ]
# matcher.add("demo", patterns)

# before_adj = []
# after_adj = []

# for post in all:
#     doc = en_nlp(post.text)
#     x = ""
#     count = 0
#     matches = matcher(doc)
#     for match_id, start, end in matches:
#         string_id = en_nlp.vocab.strings[match_id]  # Get string representation
#         span = doc[start:end]  # The matched span
#         if 'http' not in span.text and span.text not in x:
#             x = x + span.text + ", "
#             count += 1
#         # print(x + " " + str(count))
#     # Remove the last comma and space
#     post.adj = x[:-2]  # Remove the last comma and space
#     post.adj_count = count
    
#     if post.status == "Before":
#         before_adj.append(post.adj)
#     if post.status == 'After':
#         after_adj.append(post.adj)

# print(f'Before: {before_adj}')
# print(f'After: {after_adj}')



In [58]:
# csv_filename = 'Sentiment_analyzed'
# with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
#             writer = csv.writer(csv_file)
#             writer.writerow(['Status', 'Text', 'Sentiment', 'Toxicity'])
#             for post in all:
#                    writer.writerow([post.status, post.text, post.sentiment, post.toxicity, post.adj, post.adj_count])